# Pipeline for collecting explanations

In [ ]:
sample_indices = [4433, 7582, 2457, 2291, 6345, 6737, 2662, 3839, 5039, 7304, 6388, 2813, 2869, 6150, 2665, 1756, 4589, 
6286, 4001, 1596, #20
 4817, 2261, 7292, 2595, 128, 3533, 2443, 6652, 3820, 796, 3518, 110, 575, 4142, 1950, 1216, 2084, 2534, 3291, 4807, 2269,
 3625, 1154, 5049, 5653, 5924, 2366, 3425, 1821, 3610, 348, 2181, 6432, 7560, 6981, 5382, 3898, 2889, 4019, 80, 
5578, 3515, 3151, 4652, 1838, 2447, 2319, 763, 2963, 1914, 5210, 5609, 2609, 3915, 4370, 6654, 5096, 3640, 3634, 3073, 3092, 
794, 695, 4275, 757, 4527, 5194, 421, 2121, 649, 2306, 3796, 4805, 6340, 2930, 3963, 7422, 4290, 736, 900]

In [ ]:
dataset = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small.jsonl", lines=True)

In [ ]:
dataset.columns

Index(['id', 'fandoms', 'pair'], dtype='object')

In [ ]:
dataset.iloc[50285]

Int64Index([7292], dtype='int64')

In [ ]:
for count, index in enumerate(sample_indices):
    id = df['3'][index]
    index = dataset.index[dataset['id'] == id]
    print(dataset['fandoms'][index])

48333    [Star Wars Rebels, Xena: Warrior Princess]
Name: fandoms, dtype: object
33971    [Shakugan no Shana, Riddle Story of Devil/悪魔のリドル]
Name: fandoms, dtype: object
22263    [House, M.D., StarTrek: The Next Generation]
Name: fandoms, dtype: object
45410    [Batman Beyond, Flame of Recca]
Name: fandoms, dtype: object
26226    [Home Improvement, X-Men]
Name: fandoms, dtype: object
28706    [Waterloo Road, Injustice: Gods Among Us]
Name: fandoms, dtype: object
10476    [JoJo's Bizarre Adventure, Kuroko no Basuke/黒子...
Name: fandoms, dtype: object
46702    [Hyperdimension Neptunia, Silence of the Lambs...
Name: fandoms, dtype: object
945    [Final Fantasy X, Mega Man]
Name: fandoms, dtype: object
15777    [Beetlejuice, Biker Mice From Mars]
Name: fandoms, dtype: object
25514    [Biker Mice From Mars, Fushigi Yuugi]
Name: fandoms, dtype: object
28702    [Waterloo Road, Sin City]
Name: fandoms, dtype: object
39696    [21 Jump Street, In Plain Sight]
Name: fandoms, dtype: object
22231    

In [ ]:
subsamples = [0, 5, 10, 15, 20]
res = []

explainer = LimeTextExplainer(bow=True)
explainerleft = MyLimeTextExplainer(bow=True, mode='left')
explainerright = MyLimeTextExplainer(bow=True, mode='right')

for count, index in enumerate(sample_indices):


    label = df['labels'][index]
    for s in subsamples:
        cur_res = []

        correct = False
        segm = combine_segments_from_pd(index, s, 0) 
        #print(segm)
        prediction = pipeline_onesegment(segm, mode='labels')
        prediction = prediction[0]
        orig_logits = pipeline_onesegment(segm, mode='logits')
        if prediction == label:
            correct = True

        cur_res += index, s, segm, label, correct, orig_logits

        """if s == 0:
            hcf = get_head_view_avg(segm, model1, tokenizer1)
            with open("hc{}f.html".format(index), "w") as file:
                file.write(hcf.data)
            hcu = get_head_view_avg(segm, model2, tokenizer2)
            with open("hc{}u.html".format(index), "w") as file:
                file.write(hcu.data)
            for layer in range(12):
                hf = get_head_view(segm, model1, tokenizer1, layer)
                with open("h{}_{}f.html".format(index, layer), "w") as file:
                    file.write(hf.data)
                hu = get_head_view(segm, model2, tokenizer2, 11)
                with open("h{}_{}u.html".format(index, layer), "w") as file:
                    file.write(hu.data)
        """
        if prediction == 0:
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_features = exp.as_list()
            top_wordlist = []
            top_weights = []
            for f in top_features:
                top_wordlist.append(f[0])
            for word in top_wordlist:
                segm_changed = segm.replace(word, "")
                logits_changed = pipeline_onesegment(segm_changed, mode='logits')
                #print(orig_logits[0], type(orig_logits[0]), logits_changed[0], type(logits_changed[0]))
                weight = np.subtract(orig_logits[0], logits_changed[0])
                top_weights.append({word: weight})

            """for word in top_wordlist:
                segm = segm.replace(word, "")
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_sec_features = exp.as_list()
            top_sec_wordlist = []
            for f in top_sec_features:
                top_sec_wordlist.append(f[0])"""

        if prediction == 1:

            exp_l = explainerleft.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=6, num_samples=5000)
            top_features_l = exp_l.as_list()
            top_wordlist = []
            top_weights = []
            for f in top_features_l:
                top_wordlist.append(f[0])

            exp_r = explainerright.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=6, num_samples=5000)
            top_features_r = exp_r.as_list()
            for f in top_features_r:
                top_wordlist.append(f[0])
                
            for word in top_wordlist:
                segm_changed = segm.replace(word, "")
                logits_changed = pipeline_onesegment(segm_changed, mode='logits')
                #print(orig_logits[0], type(orig_logits[0]), logits_changed[0], type(logits_changed[0]))
                weight = np.subtract(orig_logits[0], logits_changed[0])
                top_weights.append({word: weight})

            """for word in top_wordlist:
                segm = segm.replace(word, "")
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_sec_features = exp.as_list()
            top_sec_wordlist = []
            for f in top_sec_features:
                top_sec_wordlist.append(f[0])"""

        cur_res += top_wordlist, top_features, top_weights

        res.append(cur_res)

        

In [ ]:
res[0]

['" Sir, I have the the reports you"ve asked for" The bridge commander stated. " Ahhh, thank you commander. Prep the fleet for the jump to Kashyyik" Admiral Segutav ordered in his thick German accent. " Yessir. Setting location now" The commander replied. " if I may ask sir, why did you need the reports on the Ghost"s crew?" " I"ve been put in charge of this fleet by none other than Grand Admiral Thrawn himself. He put me in command of this fleet with the sole purpose to destroy the Phoenix Squadron.$&*&*&$Hercules woke up when he heard Xena dreaming so he went and fetched some ale and put some sleeping poison in it. "Xena, here take a drink." Hercules kindly demanded. "Thank you, I think I need a drink." Xena replied. "Xena, what was that all about?" Hercules asked. "I had a dream I was on Mount Olympus and Ares jumped out from somewhere with Gabrielle, then we started talking and all of a sudden he killed Gabrielle." Xena sobbed.',
 0,
 True,
 array([[ 4.1582108, -4.244451 ]], dtype=

In [ ]:
segm = """" Sir, I have the the reports you"ve asked for" The bridge commander stated. " Ahhh,
 thank you commander. Prep the fleet for the jump to Kashyyik" Admiral Segutav 
ordered in his thick German accent. " Yessir. Setting location now" The commander replied. " if I may ask sir, why did you need the reports on the 
Ghost"s crew?" " 
I"ve been put in charge of this fleet by none other than Grand Admiral Thrawn himself. He put me in command of this fleet with the sole purpose to destroy the Phoenix Squadron.
$&*&*&$Hercules woke up when he heard Xena dreaming so he went
 and fetched some ale and put some sleeping poison in it. "Xena, here take a drink." Hercules kindly demanded. 
"Thank you, I think I need a drink." Xena replied. "Xena, what was that all about?" Hercules asked. 
"I had a dream I was on Mount Olympus and Ares jumped out from somewhere with Gabrielle, then we started talking and all of a sudden he killed Gabrielle." Xena sobbed."""

#segm = combine_segments_from_pd(4433, 0, 0) 
print(segm)

res = pipeline_onesegment(segm.replace("Segutav", ""), mode="logits")
res
#[ 4.5062966, -4.566842 ]
#[ 4.1582108, -4.244451 ]
#[-3.4521582,  2.2805903]

" Sir, I have the the reports you"ve asked for" The bridge commander stated. " Ahhh,
 thank you commander. Prep the fleet for the jump to Kashyyik" Admiral Segutav 
ordered in his thick German accent. " Yessir. Setting location now" The commander replied. " if I may ask sir, why did you need the reports on the 
Ghost"s crew?" " 
I"ve been put in charge of this fleet by none other than Grand Admiral Thrawn himself. He put me in command of this fleet with the sole purpose to destroy the Phoenix Squadron.
$&*&*&$Hercules woke up when he heard Xena dreaming so he went
 and fetched some ale and put some sleeping poison in it. "Xena, here take a drink." Hercules kindly demanded. 
"Thank you, I think I need a drink." Xena replied. "Xena, what was that all about?" Hercules asked. 
"I had a dream I was on Mount Olympus and Ares jumped out from somewhere with Gabrielle, then we started talking and all of a sudden he killed Gabrielle." Xena sobbed.
Tokenization...


1ex [00:00, 999.60ex/s]


Obtaining embeddings...
> processing item 0/1
Done!


array([[-3.4521582,  2.2805903]], dtype=float32)

In [ ]:
respd = pd.DataFrame (res, columns = ['text', 'label', 'correctness', 'logits', 'topwords', 'topwords_lime', 'topwords_obf'])

In [ ]:
respd.to_csv('res_100.csv', index=False)

In [ ]:
res[0][4]

[{'Segutav': array([ 7.6103687, -6.5250416], dtype=float32)},
 {'Xena': array([-1.1145916,  1.1298003], dtype=float32)},
 {'Ahhh': array([-0.49818182,  0.47477627], dtype=float32)},
 {'I': array([ 0.6196306, -0.551337 ], dtype=float32)},
 {'Hercules': array([-0.6051874 ,  0.58823967], dtype=float32)},
 {'Sir': array([-0.6503434,  0.6286473], dtype=float32)},
 {'now': array([-0.01462936,  0.0202775 ], dtype=float32)},
 {'ordered': array([ 0.7938595, -0.7076478], dtype=float32)},
 {'Thank': array([ 0.28726006, -0.29658556], dtype=float32)},
 {'replied': array([-1.3187771,  1.4551167], dtype=float32)}]

In [ ]:
predictions_onetext = pipeline_onetext(text4)

In [ ]:
def get_data_from_combined_texts_long(text_or_list):
    """
    Get data from raw text that contains two fragments and a separater, or from a list of texts,
    each of them containing two fragments and a separater. Used in pipeline_onetext. The ONLY type
    of data processor for LIME inputs
    """

    print("Getting data from raw text")

    datas = []

    print(type(text_or_list), len(text_or_list))
    if not isinstance(text_or_list, str):
        for text_variant in text_or_list:
            text1, text2 = text_variant.split("$&*&*&$")
            text1 = text_segmentate(text1, maxlen=750, seps='.?!;')
            text2 = text_segmentate(text2, maxlen=750, seps='.?!;')
            while len(text1) < 30 or len(text2) < 30:
                    if len(text1) < 30:
                        n_text1 = []
                        for i in range(30):
                            for sent in text1:
                                n_text1.append(sent)
                        text1 = n_text1
                    elif len(text2) < 30:
                        n_text2 = []
                        for i in range(30):
                            for sent in text2:
                                n_text2.append(sent)
                        text2 = n_text2
            datas.append((text1, text2))
    else:
        text1, text2 = text_or_list.split("$&*&*&$")
        text1 = text_segmentate(text1, maxlen=750, seps='.?!;')
        text2 = text_segmentate(text2, maxlen=750, seps='.?!;')
        while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
        datas.append((text1, text2))
    return datas


In [ ]:
with open('textcomb3.txt', 'r') as text:
    text3 = text.read()

segments = get_data_from_combined_texts_long(text3)

Getting data from raw text
<class 'str'> 42413


In [ ]:
def combine_segments(index, sep_option=0, write=False):
    """
    Combine a pair of texts from dataset with a separator and turn into a single text
    """

    text1 = segments[0][0][index]
    text2 = segments[0][1][index]
    sep = "$&*&*&$" if sep_option == 0 else "[SEP]"
    text_combined = text1 + sep + text2
    
    if write:
        name = "textcomb{}.txt".format(index)
        with open(name, 'w') as textcomb:
            textcomb.write(text_combined)

    return(text_combined)

In [ ]:
segm2 = combine_segments(1)
segm2

'Rinoa noticed how there were so much dishes served, enough to feed one whole army. "Please, take a seat." Laguna gestured at them as he sat at a chair placed at the end of the long table. He then looked around, as if looking for someone. "Your son said he"ll be joining us in a few minutes." Kiros plainly said. "Ah..." Laguna gave out a sheepish smile. "Well, can"t let our guests wait too long, can we? C"mon now, gobble up! Don"t be shy, just eat...$&*&*&$She even almost succeeded at her own assassination attempt at the crown prince, only to pull back and used a lame excuse as "I do not do overtimes." when the battle ran for too long. But she did also express her intentions to leave her post as their hired mercenary afterwards when things got too senseless and chaotic with the Imperials. She was only in for the job and money, not for some loyalty or building a good name in her trade.'

In [ ]:
segm1 = combine_segments(0, 1)
segm1

'"DNA?" Javier asked. The ME met his gaze, eyes twinkling. "Oh yeah. And lots of it." "Oh, I"m listening," Ryan said with a grin. "Well, there were male epithelials under her fabulously manicured nails, but I"ve got at least three other donors." Ryan and Javier exchanged a look. "Define donors," Ryan requested. "I"ve got saliva and semen," Lanie replied, checking the notes in the folder. "Multiple donors for both too. From the looks of things our girl had multiple sexual partners just before she died." Ryan"s eyebrow went up. "Are you telling us this girl was in an orgy?" "Dude, Castle would love this," Javier couldn"t help exclaiming. He shook his head as he and Ryan exchanged a look.[SEP]"Are you insane?" She"d subconsciously taken a step closer to him. However, he mirrored her step backwards. "I"m not finished. You deserve more answers than that." This new found honesty shocked Sara enough to keep her mouth shut. However, he stopped, rubbing a hand over his face and sighing, partly 

In [ ]:
testtext16 = combine_segments_from_pd(0,16)
testtext16

'What hospital are you at?" Dustin said and there was a lot of rustling at the other end. "The one near my house. I didn"t catch the name I was a little occupied. Mary was throwing a fit." Joey answered sincerely. "Kay, we are coming. See you in a few." Dustin said and the line went dead. Joey took a seat along the wall and waited for news about Mary. Just a tad bit of drama, hope its not too confusing. Blondie : P Oh gosh, sorry. I left you a cliff hanger. I wonder what will happen to Mary today.$&*&*&$She bobbed her head slightly from side to side and with a sheepish look remarked, "Not exactly but something to the effect. I did call him a ninny though." John flashed his killer smile, it made her gooey every time. In an instant his eyes turned intense again and he rested his forehead on hers. The change in him didn"t go unnoticed by Clarice. "What?" "Clarice I...feel it again." He had told her about how he sometimes felt a foreboding feeling. "Hey.'

In [ ]:
testtext0_changed

'"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." Mary yelled as she continued to yell and jump about.\n "Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. "Wow your lighter \n then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.$&*&*&$"Are you okay?" His simple inquiry about her \n farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment. "What do you think?" her face now  the \nevidence of the strain she had been carrying for weeks. Her separation from Marcos made her achy and distressed. Her heart was breaking into a million\n pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him.'

In [ ]:
testtext0_changed = """"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." MARY yelled as she continued to yell and jump about.
 "Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. "Wow your lighter 
 then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.$&*&*&$"Are you okay?" His simple inquiry about her 
 farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment. "What do you think?" her face now  the 
evidence of the strain she had been carrying for weeks. Her separation from Marcos made her achy and distressed. Her heart was breaking into a million
 pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

"""She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, she reached 
down and pulled out the spare key Joey kept there for her. She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs 
into his room. She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house. "Mary!" Joey yelled, standing 
dumb-struck in the doorway.$&*&*&$"Joh..n." her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. 
They jerked apart and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. 
Then she bit out,"Nice going laser. As usual I"ll have to fix your mess." Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it."""

'She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, she reached \ndown and pulled out the spare key Joey kept there for her. She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs \ninto his room. She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house. "Mary!" Joey yelled, standing \ndumb-struck in the doorway.$&*&*&$"Joh..n." her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. \nThey jerked apart and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. \nThen she bit out,"Nice going laser. As usual I"ll have to fix your mess." Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it.'

In [ ]:
res = pipeline_onesegment(testtext16, mode='logits')
res

Getting data from raw text
<class 'str'> 963
Tokenization...


1ex [00:00, 999.12ex/s]


Obtaining embeddings...
> processing item 0/1
Done!


array([[-5.0536942,  4.766159 ]], dtype=float32)

In [ ]:
testtext0_changed = testtext16.replace("Clarice","")


res = pipeline_onesegment(testtext0_changed, mode='logits')
print(res)


#example 0-0 
#[[ 4.7861366 -4.759954 ]] class 0 correct confidence 1
#['She', 'MARY', 'and', 'proceeded', 'ran', 'bush']
#without She (weight 0.28) [[ 4.1627555 -4.1556478]]
#without MARY (weight 0.24) [[ 4.9938765 -5.0127206]] contrary to Lime!
#if we replace MARY with Mary the difference becomes larger [[ 5.4506316 -5.633418 ]] which means that CASING MATTERS
#if we replace MARY with JOHN texts get even more different [[ 5.380384 -5.513166]]. Keep increasing slightly if we add more JOHN
#if we add JOHN to the second part as well, gets less different [[-4.439573  3.488577]]
#without all topwords [[-2.9259872  1.8621671]] LABEL CHANGES

#example 0-4
#[ 1.371373 , -1.5087308] class 0 correct confidence 0.95
#['bore', 'yelled', 'strain', 'worded', 'What', 'said']
# without bore (weight 0.15) label SUDDENLY CHANGES [[-2.2827718  1.4593749]]
# without worded (weight 0.14) label ALSO SUDDENLY CHANGES [[-1.9648905  1.2226754]]
# without farewell (weight 0.12) ALSO [[-2.2466617  1.45032  ]], same for touched, said
# and they somewhat combine: when we remove all 3 -- bigger changer [[-4.2394676  3.2303755]]
# while without yelled (weight 0.11) it only goes in another direction [[ 2.6601286 -2.9694483]]
# SO here the model very easily switches to class 1

#example 0-16
#[[-5.0536942,  4.766159 ]] class 1 incorrect confidence 1
# however, it's enough to remove 1 name. Without Clarice (weight 0.26) [[ 2.7647822 -3.1579168]]


Getting data from raw text
<class 'str'> 949
Tokenization...


1ex [00:00, 499.32ex/s]


Obtaining embeddings...
> processing item 0/1
Done!
[[ 2.7647822 -3.1579168]]


In [ ]:
segm1 = """Rinoa let out a soft giggle. "Okay Uncle Laguna." "As always, make yourselves at home!" Kiros cleared his throat. "Laguna, I believe our guests are hungry." "OH! Yes yes, I"m sorry." 
Laguna scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed Laguna as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one, though the lenghty table was pleasingly decorated with foods on the table.$&*&*&$The crown prince already entreated help from the glaives, a last resort he would 
rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none.
 It was when nea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""

In [ ]:
segm3 = """"" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." Mary yelled as she continued to yell and jump about. 
"Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. 
"Wow your lighter then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.$&*&*&$"Are you okay?" 
His simple inquiry about her farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment.
 "What do you think?" her face now bore the evidence of the strain she had been carrying for weeks. 
Her separation from Marcos made her achy and distressed. Her heart was breaking into a million pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

In [ ]:
segm4 = """What hospital are you at?" Dustin said and there was a lot of rustling at the other end. "The one near my house, Clarice. I didn"t catch the name I was a little occupied. 
Mary was throwing a fit." Joey answered sincerely. "Kay, we are coming. See you in a few." Dustin said and the line went dead. Joey took a seat along the wall and waited for news about Mary.
 Just a tad bit of drama, hope its not too confusing. Blondie : P Oh gosh, sorry. I left you a cliff hanger. I wonder what will happen to Mary today.$&*&*&$Clarice bobbed her head slightly from side to side and with a sheepish look remarked,
  "Not exactly but something to the effect, Clarice. I did call him a ninny though." John flashed his killer smile, it made her gooey every time. In an instant his eyes turned intense again and he rested his forehead on hers. The change in him didn"t go unnoticed by Clarice. 
"What?" "Clarice I...feel it again." He had told her about how he sometimes felt a foreboding feeling. "Hey."""

In [ ]:
res = pipeline_onesegment(segm1, mode='logits')
res = [-5.531711 ,  5.7830925], [-5.3570204,  5.4185996]
#Without Ara
#shap prediction 1: 0.506, 0: -0.474
#in reality 1: 0.364, 0: -0.175

Getting data from raw text
<class 'str'> 929
Tokenization...


1ex [00:00, 998.41ex/s]


Obtaining embeddings...
> processing item 0/1
Done!


array([[-5.531711 ,  5.7830925]], dtype=float32)

In [ ]:
res = pipeline_onesegment(segm4, mode='logits')
res
#[-5.0536942,  4.766159 ]
#One more Clarice [-5.1230087,  4.9003825]
#Two more Clarice[-5.2671213,  5.1808963]
#Third Clarice now in the first segment [-5.4629855,  5.5636225]

Getting data from raw text
<class 'str'> 990
Tokenization...


1ex [00:00, 999.60ex/s]


Obtaining embeddings...
> processing item 0/1
Done!


array([[-5.4629855,  5.5636225]], dtype=float32)

In [ ]:
res = pipeline_onesegment(segm3, mode='logits')
res
#[ 1.371373 , -1.5087308]
#OW [ 2.905075 , -3.2013812]
#! [ 2.2600317, -2.5298123]
#OW! [-0.00994844, -0.42599773]
#shap prediction OW for class 1 OW -0.305 ! -0.27
#shap prediction OW for class 0 OW 0.303 ! 0.308
#in reality for class 1 OW +1.54 ! +1.02 OW! -1.08
#in reality for class 0 OW -1.53 ! -0.89 OW! 1.38 


Getting data from raw text
<class 'str'> 987
Tokenization...


1ex [00:00, 999.36ex/s]


Obtaining embeddings...
> processing item 0/1
Done!


array([[-0.00994844, -0.42599773]], dtype=float32)

In [ ]:
segm1

'Rinoa let out a soft giggle. "Okay Uncle Laguna." "As always, make yourselves at home!" Kiros cleared his throat. "Laguna, I believe our guests are hungry." "OH! Yes yes, I"m sorry." Laguna scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed Laguna as he went through one of the sliding doors on the right side of the room. The dining hall was a plain one, though the lenghty table was pleasingly decorated with foods on the table.$&*&*&$The crown prince already entreated help from the glaives, a last resort he would rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none. It was when Aranea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders.'